In [1]:
! pip list

Package                   Version
------------------------- --------------
anyio                     4.7.0
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asttokens                 3.0.0
async-lru                 2.0.4
attrs                     24.2.0
babel                     2.16.0
beautifulsoup4            4.12.3
bleach                    6.2.0
certifi                   2024.8.30
cffi                      1.17.1
charset-normalizer        3.4.0
colorama                  0.4.6
comm                      0.2.2
contourpy                 1.3.1
cycler                    0.12.1
debugpy                   1.8.9
decorator                 5.1.1
defusedxml                0.7.1
executing                 2.1.0
fastjsonschema            2.21.1
filelock                  3.13.1
fonttools                 4.55.3
fqdn                      1.5.1
fsspec                    2024.2.0
gitdb                     4.0.11
GitPython                 3.1.43
h11       

In [10]:
!pip install -r requirements.txt

  Using cached cryptography-44.0.0-cp39-abi3-manylinux_2_28_x86_64.whl.metadata (5.7 kB)
Reason for being yanked: This version accidentally included an unintentional GPL dependency. Please upgrade to 0.4.2.
Reason for being yanked: This version accidentally included an unintentional GPL dependency. Please upgrade to 0.4.2.
Reason for being yanked: This version accidentally included an unintentional GPL dependency. Please upgrade to 0.4.2.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 58.3 MB/s eta 0:00:00:00:01
Using cached cryptography-44.0.0-cp39-abi3-manylinux_2_28_x86_64.whl (4.2 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.9/796.9 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 42.1 MB/s eta 0:00:00
  Attempting uninstall: stack-data
    Found existing installation: stack-data 0.6

In [15]:
!pip install azure-identity

In [28]:
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import CodeInterpreterTool
from azure.identity import DefaultAzureCredential
import os

In [35]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str='eastus2.api.azureml.ms;ff29343b-3d73-44c7-b0ce-9f33d4a50a7a;rg-genaiops-dev;ai-project-butrkhj3nqgog',
)

In [36]:

with project_client:

    code_interpreter = CodeInterpreterTool()
    
    agent = project_client.agents.create_agent(
        model="gpt-4o-mini",
        name="my-agent",
        instructions="You are helpful agent",
        tools=code_interpreter.definitions,
        # tool_resources=code_interpreter.resources,
    )

    thread = project_client.agents.create_thread()

    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content="""
        
        You are my Python programming assistant. Generate code and execute it according to the following requirements

        1. Save "this is blog" to blog-{YYMMDDHHMMSS}.md

        2. give me the download this file link
            """,
    )
    # create and execute a run
    run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        # Check if you got "Rate limit is exceeded.", then you want to get more quota
        print(f"Run failed: {run.last_error}")

    # # delete the original file from the agent to free up space (note: this does not delete your version of the file)
    # project_client.agents.delete_file(file.id)
    # print("Deleted file")

    # print the messages from the agent
    messages = project_client.agents.get_messages(thread_id=thread.id)
    print(f"Messages: {messages}")

    # get the most recent message from the assistant
    last_msg = messages.get_last_text_message_by_sender("assistant")
    if last_msg:
        print(f"Last Message: {last_msg.text.value}")

    # print(f"File: {messages.file_path_annotations}")


    for file_path_annotation in messages.file_path_annotations:

        file_name = os.path.basename(file_path_annotation.text)

        project_client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name,target_dir="./blog")


    

Run finished with status: RunStatus.COMPLETED
Messages: <azure.ai.projects.models._patch.ThreadMessages object at 0x727c9e7ff470>
Last Message: The file has been created successfully. You can download it using the link below:

[Download blog-250116182914.md](sandbox:/mnt/data/blog-250116182914.md)
